In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import norm
from scipy import stats
from scipy.integrate import quad
import pandas as pd
from scipy.stats import norm

# Case 5: Multi-Look and American Options

## Part 1: Asian Options

In [4]:
S0 = 100 
r = 0.02 
sigma = 0.15
T = 10  # Time horizon
n = 10  # Number of observations
n_sims = 100000  # Number of simulation paths
guarantee = 100

In [9]:
def AsianOptionMonteCarlo(S0, T, r, sigma, numSteps, numSim):
    dt = T / numSteps 

    # Stock price paths
    S = np.zeros((numSim, numSteps + 1))  
    S[:, 0] = S0  

    Z = np.random.normal(0, 1, (numSim, numSteps)) 
    for t in range(1, numSteps + 1):
        S[:, t] = S[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, t - 1])

    # Calculate payoff: max(average stock price - 100, 0)
    average_price = np.mean(S[:, 1:], axis=1)  # Exclude S[:, 0] (initial price)
    payoff = np.maximum(average_price - 100, 0)

    discountedPrice = np.exp(-r * T) * np.mean(payoff)
    SE = np.exp(-r * T) * np.std(payoff) / np.sqrt(numSim)

    return discountedPrice, SE

asianOption_price, SE = AsianOptionMonteCarlo(S0, T, r, sigma, numSteps=10, numSim=100000)

print("Price of the Asian contract: ", asianOption_price, "±", SE)

Price of the Asian contract:  15.683419800847233 ± 0.07181925645715698


In [ ]:
# Second exercise
def asian_option_price(S0, r, sigma, T, K):

    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0 / K) + (r - 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    # Price using Black-Scholes
    phi = norm.cdf
    EU_price = S0 * phi(d1) - np.exp(-r * T) * K * phi(d2)
    return EU_price

print(asian_option_price(S0, r, sigma, T=10, K=100))

27.571349248747218


# Adapted Black Scholes

In [7]:
# Last part
def compute_moments(S_0, r, sigma, T):
    # Number of time steps
    N = 10
    
    # First moment: E_Q[A]
    E_Q_A = (1 / N) * sum(S_0 * np.exp(r * np.arange(1, N + 1)))
    
    # Second moment: E_Q[A^2]
    E_Q_A2 = (1 / N**2) * sum(
        sum(S_0**2 * np.exp(r * (i + np.arange(1, N + 1)) + sigma**2 * np.minimum(i, np.arange(1, N + 1))))
        for i in range(1, N + 1)
    )
    
    # Compute A0 and b
    A0 = E_Q_A / np.exp(r * T)
    b = np.sqrt((1 / T) * np.log(E_Q_A2 / (A0**2)) - 2 * r)
    
    return A0, b

def asian_option_price(S0, r, sigma, T, K):
    moments = compute_moments(S0, r, sigma, T)
    A0, b = moments
    
    # Compute d1 and d2
    d1 = (np.log(A0 / K) + (r + 0.5 * b**2) * T) / (b * np.sqrt(T))
    d2 = (np.log(A0 / K) + (r - 0.5 * b**2) * T) / (b * np.sqrt(T))
    
    # Black-Scholes price
    phi = norm.cdf
    BS_price = A0 * phi(d1) - np.exp(-r * T) * K * phi(d2)
    return BS_price

# Example usage
S_0 = 100   # Initial stock price
r = 0.02    # Risk-free rate
sigma = 0.15  # Volatility
T = 10
K = 100

print(asian_option_price(S_0, r, sigma, T, K))

15.973823843043952


## Part 2: Unit-linked with continuous guarantee